In [ ]:
#Importovanje neophodnih modula iz biblioteke Transformers

from transformers import GPT2LMHeadModel , GPT2Tokenizer 

#Importovanje modula Translator

from googletrans import Translator

In [ ]:
#Inicijalizacija GPT-2 tokenizatora sa 'gpt2-large' obučenim modelom

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large') 

#Inicijalizacija GPT-2 jezičkog modela sa 'gpt2-large' obučenim modelom 

model = GPT2LMHeadModel.from_pretrained('gpt2-large' , pad_token_id = tokenizer.eos_token_id)

In [ ]:
#Funkcija za generisanje teksta

def generate_text(input_text,source_language):
    
    # Model efikasno generise tekst na engleskom jeziku, aко је rečenica napisana u nekom drugom jeziku, prevodi se
    
    if source_language != 'en':
        input_text = translate(input_text, source_language, 'en')

    #Enkodiranje unesene rečenice u PyTorch tenzore (niz cijelih brojeva)
    
    input_ids = tokenizer.encode(input_text, return_tensors='pt') 
    
    # primjer: 'tensor([[ 2061,   318,  8300,  3303, 11361]])'
    
    #Generisanje novog teksta koristeći GPT-2 model
    
    #Parametri metode generate:
    # max_length - maksimalan broj rijeci,
    # num_beams - genrise se 5 uzastopnih rijeci sa najvecim prioritetom,
    # no_repeat_ngram_size - ista sekvenca se ne ponavlja vise od 2 puta,
    # early_stopping - ukoliko dodje do ponavljanja sekvence, generisanje se prekida
    
    output = model.generate(input_ids, max_length = 100, num_beams = 5,no_repeat_ngram_size = 2,early_stopping = True)
    
    #Dekodiranje generisanog izlaza
    
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    ispis(decoded_output, source_language)

In [ ]:
#Funkcija za prevodjenje teksta

def translate(input, src, dest):
    translator = Translator()
    translated_text = translator.translate(input, src=src, dest=dest)
    return translated_text.text

In [ ]:
#Funkcija za ispis generisanog teksta

def ispis(decoded_output, source_language):
    # Ukoliko tekst ne završаva tačkom, radi ljepšeg ispisa
    
    if decoded_output.endswith('.'):
        pass
    else:
        # Ako posljednja rečenica ne sadrži tačku, briše se
        last_period_index = decoded_output.rfind('.')
        if last_period_index != -1:
            decoded_output = decoded_output[:last_period_index + 1]   
    
    # Prevodjenje teksta u počеtni jezik 
    
    if source_language != 'en':
        decoded_output = translate(decoded_output, 'en', source_language)

    # Ispisivanje generisanog teksta
    
    # Može se desiti greška: 'QUERY LENGTH LIMIT EXCEEDED', kada je dužina ispisa veća od maksimalne dužine koju je naveo API
    # Zbog toga, dijenje teksta u manje cjeline
    if(len(decoded_output) >= 500):
        parts = [decoded_output[i:i+500] for i in range(0, len(decoded_output), 500)]
        for part in parts:
            print(part, end = '')
    else:
        print(decoded_output)

In [ ]:
#Definisanje teme/pitanja i generisanje novog teksta

if __name__ == "__main__":
    input_text = 'Sta je programerski jezik Python?' 
    source_language = 'sr'  
    generate_text(input_text, source_language)